In [1]:
import numpy as np
import requests
import datetime
import pandas as pd
import re
import json
from datetime import date, timedelta
from datetime import datetime
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr,first, last,when, split, col,lit, concat, date_format,to_utc_timestamp,to_timestamp, regexp_replace,concat_ws
from google.cloud import bigquery
client = bigquery.Client(location="us-central1")
#print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)
pd.set_option("max_colwidth",10000)
pd.set_option("max_rows",1000)
pd.set_option("max_columns",100)

In [2]:
spark = SparkSession.builder.appName("Sandbox_Allocation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/03 19:06:02 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/03 19:06:02 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/03 19:06:02 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/03 19:06:02 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
#current_date = datetime.today()
#dias_atras = 1
#str_day = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")

STR_YEARMONTH = "2023-04"
day_ini_1 = '2023-04-01'
day_fin_1 = '2023-04-30'
day_fin_2 = '2023-05-01'
print(day_ini_1,day_fin_1)

2023-04-01 2023-04-30


## Sites to get tenant Quantity

In [4]:
df_sites_yts1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','databricks.saas-sites')\
.option("filter","date(timestamp) between '%s' and '%s' and environment = 'yts1'"%(day_ini_1,day_fin_1))\
.load()
df_sites_yts1 = df_sites_yts1.groupBy('tenant_parent_code').count()
df_sites_yts1 = df_sites_yts1[['tenant_parent_code']]
df_sites_yts1.show()

+------------------+
|tenant_parent_code|
+------------------+
|              CENT|
|              FALA|
|            CELINE|
|               PEI|
|            FALACO|
|               CDC|
|              OECH|
|               CSU|
|               TRR|
|             PR802|
|               PRN|
|              RSTQ|
|            LEROYM|
|               QSR|
|            FALAPE|
|              CHIP|
|           STUDIOF|
|             TOSCA|
|            JFRESH|
|             FISIA|
+------------------+
only showing top 20 rows



In [5]:
df_sites_vts1 = spark.read.format('bigquery')\
.option('project','saas-analytics-io')\
.option('table','databricks.saas-sites')\
.option("filter","date(timestamp) between '%s' and '%s' and environment = 'vts1'"%(day_ini_1,day_fin_1))\
.load()
df_sites_vts1 = df_sites_vts1.groupBy('tenant_parent_code').count()
df_sites_vts1 = df_sites_vts1[['tenant_parent_code']]
df_sites_vts1.show()

+------------------+
|tenant_parent_code|
+------------------+
|              NAVY|
|             HONDA|
|             FEDEX|
|             MOJIX|
|              IKEA|
+------------------+



## YTS1 GCP Cost

In [ ]:
#df_vt1_gcp = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','yts1_v1data1.metadata').option("filter", """date(usage_start_time) between '%s' and '%s'"""%(day_ini_1,day_fin_1)).load()

In [6]:
query = """
    SELECT service,sku,usage_start_time, cost, usage, credits
    FROM `saas-analytics-io.finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D` 
    WHERE JSON_VALUE(project.number) = '600940915297' 
    and date(usage_start_time) between '%s' and '%s' 
    --and usage_start_time between '2022-09-08 00:00:00' and '2022-09-28 23:59:59'  """ %(day_ini_1,day_fin_2)

query_job = client.query(
    query,
# Location must match that of the dataset(s) referenced in the query.
    location = "US",
)  # API request - starts the query
print(query)
df_yt1_gcp = query_job.to_dataframe()


    SELECT service,sku,usage_start_time, cost, usage, credits
    FROM `saas-analytics-io.finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D` 
    WHERE JSON_VALUE(project.number) = '600940915297' 
    and date(usage_start_time) between '2023-04-01' and '2023-05-01' 
    --and usage_start_time between '2022-09-08 00:00:00' and '2022-09-28 23:59:59'  


In [7]:
print("Total Registros metadata ", df_yt1_gcp.count())

Total Registros metadata  service             217239
sku                 217239
usage_start_time    217239
cost                217239
usage               217239
credits             217239
dtype: int64


## VTS1 GCP Cost

In [8]:
#df_vt1_gcp = spark.read.format('bigquery').option('project','saas-analytics-io').option('table','yts1_v1data1.metadata').option("filter", """date(time) between '%s' and '%s'"""%(day_ini_1,day_fin_1)).load()

In [9]:
query = """
    SELECT service,sku,usage_start_time, cost, usage, credits
    FROM `saas-analytics-io.finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D` 
    WHERE JSON_VALUE(project.number) = '361820198261' 
    and date(usage_start_time) between '%s' and '%s' 
    --and usage_start_time between '2022-09-08 00:00:00' and '2022-09-28 23:59:59'  """%(day_ini_1,day_fin_2)

query_job = client.query(
    query,
# Location must match that of the dataset(s) referenced in the query.
    location = "US",
)  # API request - starts the query
print(query)
df_vt1_gcp = query_job.to_dataframe()


    SELECT service,sku,usage_start_time, cost, usage, credits
    FROM `saas-analytics-io.finance.gcp_billing_export_v1_01FE23_B1D2D8_10D34D` 
    WHERE JSON_VALUE(project.number) = '361820198261' 
    and date(usage_start_time) between '2023-04-01' and '2023-05-01' 
    --and usage_start_time between '2022-09-08 00:00:00' and '2022-09-28 23:59:59'  


In [10]:
print("Total Registros metadata ", df_vt1_gcp.count())

Total Registros metadata  service             101368
sku                 101368
usage_start_time    101368
cost                101368
usage               101368
credits             101368
dtype: int64


## Data Preparation

In [11]:
df_sites_yts1.loc[df_sites_yts1['tenant_parent_code'].isin(['CENT','PERN','PRN','FISIA','CDC','CSU','TOSCA','FALACO','FALAPE','FALA','CHIP','RSTQ','PEI','IKEA','CELINE','JFRESH','TRR','LEROYM','OECH']),'valid_tenant']=1
df_sites_vts1.loc[df_sites_vts1['tenant_parent_code'].isin(['BOLZ','IKEA','FEDEX','HONDA']),'valid_tenant']=1

AttributeError: 'DataFrame' object has no attribute 'loc'

In [ ]:
#df_sites_yts1
df_sites_yts1[df_sites_yts1['valid_tenant'].isnull()]

Out[15]:

,tenant_parent_code,valid_tenant
2,MRGRT,NaN
15,QSR,NaN


In [ ]:
df_sites_vts1[df_sites_vts1['valid_tenant'].isnull()]

Out[16]:

,tenant_parent_code,valid_tenant
0,NAVY,NaN
3,MOJIX,NaN


In [ ]:
tenant_qty_yt1 = df_sites_yts1['valid_tenant'].count()
tenant_qty_vt1 = df_sites_vts1['valid_tenant'].count()

In [ ]:
df_sites_yts1.loc[:,'yearmonth'] = STR_YEARMONTH
df_sites_yts1_s = df_sites_yts1[df_sites_yts1['valid_tenant']==1] 

In [ ]:
df_sites_vts1.loc[:,'yearmonth'] = STR_YEARMONTH
df_sites_vts1_s = df_sites_vts1[df_sites_vts1['valid_tenant']==1] 

In [ ]:
print(tenant_qty_yt1)
print(tenant_qty_vt1)

18
4

In [ ]:
df_yt1_gcp.loc[:,'usage_start_time_LA']= df_yt1_gcp['usage_start_time'].dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
df_vt1_gcp.loc[:,'usage_start_time_LA']= df_vt1_gcp['usage_start_time'].dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)

In [ ]:
df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("[","")
df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("]","")
df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("","")
try:
    df_yt1_gcp.loc[df_yt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = df_yt1_gcp['credits'].str.split('\n', expand=True)[0]
    df_yt1_gcp.loc[df_yt1_gcp['credits_1'].isnull(),'credits_1']=''
    json_cols = ['service', 'sku','usage','credits_1']
    try:
        df_yt1_gcp.loc[df_yt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_2'] = df_yt1_gcp['credits'].str.split('\n', expand=True)[1]
        df_yt1_gcp.loc[df_yt1_gcp['credits_2'].isnull(),'credits_2']=''
        json_cols = ['service', 'sku','usage','credits_1','credits_2']
    except:
        print ('2')
        json_cols = ['service', 'sku','usage','credits_1']
except:
    df_yt1_gcp.loc[df_yt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = df_yt1_gcp['credits'][0]
    df_yt1_gcp.loc[df_yt1_gcp['credits_1'].isnull(),'credits_1']=''
    json_cols = ['service', 'sku','usage','credits_1']


sku = df_yt1_gcp['sku'].values.tolist()
service = df_yt1_gcp['service'].values.tolist()

#json_cols = ['service', 'sku','usage','credits_1','credits_2']
for column in json_cols:
    #c_load = test[column].apply(json.loads)
    
    if(column == 'project'):  
        pr = pd.json_normalize(df_yt1_gcp['project'])
        pr.rename(columns = {'labels':'labels_%s'%(column)},inplace =True)
        df_yt1_gcp = df_yt1_gcp.join(pr)
        df_yt1_gcp.rename(columns = {'id':'id_%s'%(column)},inplace =True)
        df_yt1_gcp.rename(columns = {'labels':'description_%s'%(column)},inplace =True)
        df_yt1_gcp = df_yt1_gcp.drop(column , axis=1)
    if((column=='credits_1')|(column== 'credits_2')):
        #df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("[","")
        #df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("]","")
        #df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("","")
        df_yt1_gcp.loc[~(df_yt1_gcp[column].str.contains('1|2|3|4|5|6|7|8|9|0')),column] = "{'name':'','amount':0,'full_name':'','id':'', 'type':''}"
        t_er = list(df_yt1_gcp[column])
        t_er2 = json.dumps(t_er)
        t_er2 = t_er2.replace('"',"").replace("'",'"')
        if column == 'credits_1':
            df_credits = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits.rename(columns ={'amount':'amount_credits_1','id':'id_credits_1','name':'name_credits_1'
                                   ,'full_name':'full_name_credits_1', 'type':'type_credits_1'},inplace = True)
            df_yt1_gcp = df_yt1_gcp.join(df_credits)
            df_yt1_gcp.loc[df_yt1_gcp['type_credits_1']=='PROMOTION','promotion_credits_1'] = df_yt1_gcp['amount_credits_1']
            df_yt1_gcp.loc[df_yt1_gcp['type_credits_1']=='PROMOTION','amount_credits_1'] = 0
        if column == 'credits_2':
            df_credits_2 = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits_2.rename(columns ={'amount':'amount_credits_2','id':'id_credits_2','name':'name_credits_2'
                                   ,'full_name':'full_name_credits_2', 'type':'type_credits_2'},inplace = True)
            df_yt1_gcp = df_yt1_gcp.join(df_credits_2)
            df_yt1_gcp.loc[df_yt1_gcp['type_credits_2']=='PROMOTION','promotion_credits_2'] = df_yt1_gcp['amount_credits_2']
            df_yt1_gcp.loc[df_yt1_gcp['type_credits_2']=='PROMOTION','amount_credits_2'] = 0
        
        df_yt1_gcp = df_yt1_gcp.drop(column , axis=1)

    if((column != 'project')&(column != 'credits_1')&(column != 'credits_2')):
        c_list = list(df_yt1_gcp[column])
        c_dat = json.dumps(c_list)    
        df_yt1_gcp = df_yt1_gcp.join(pd.read_json(c_dat))
        df_yt1_gcp.rename(columns = {'id':'id_%s'%(column)},inplace =True)
        df_yt1_gcp.rename(columns = {'description':'description_%s'%(column)},inplace =True)
        df_yt1_gcp = df_yt1_gcp.drop(column , axis=1)
try:        
  df_yt1_gcp.loc[:,'total_after_discounts']=  df_yt1_gcp['cost'] + df_yt1_gcp['amount_credits_1']+ df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[:,'amount_credits']= df_yt1_gcp['amount_credits_1']+ df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[(~(df_yt1_gcp['description_sku'].str.contains('SSD'))),'processing_discount']= df_yt1_gcp['amount_credits_1']+df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[(df_yt1_gcp['description_sku'].str.contains('SSD')),'storage_discount']= df_yt1_gcp['amount_credits_1']+df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[(~(df_yt1_gcp['description_sku'].str.contains('SSD'))),'total_processing_after_discount']=  df_yt1_gcp['cost']+df_yt1_gcp['amount_credits_1']+ df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[((df_yt1_gcp['description_sku'].str.contains('SSD'))),'total_storage_after_discount']=  df_yt1_gcp['cost']+df_yt1_gcp['amount_credits_1']+ df_yt1_gcp['amount_credits_2']
  df_yt1_gcp.loc[:,'promotion_credits'] = df_yt1_gcp['promotion_credits_1'].fillna(0) + df_yt1_gcp['promotion_credits_2'].fillna(0)
except:
  df_yt1_gcp.loc[:,'total_after_discounts']=  df_yt1_gcp['cost'] + df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[:,'amount_credits']= df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[(~(df_yt1_gcp['description_sku'].str.contains('SSD'))),'processing_discount']= df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[(df_yt1_gcp['description_sku'].str.contains('SSD')),'storage_discount']= df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[(~(df_yt1_gcp['description_sku'].str.contains('SSD'))),'total_processing_after_discount']=  df_yt1_gcp['cost']+df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[((df_yt1_gcp['description_sku'].str.contains('SSD'))),'total_storage_after_discount']=  df_yt1_gcp['cost']+df_yt1_gcp['amount_credits_1']
  df_yt1_gcp.loc[:,'promotion_credits'] = df_yt1_gcp['promotion_credits_1'].fillna(0)

df_yt1_gcp.loc[(~(df_yt1_gcp['description_sku'].str.contains('SSD'))),'processing_cost']=  df_yt1_gcp['cost']
df_yt1_gcp.loc[(df_yt1_gcp['description_sku'].str.contains('SSD')),'storage_cost']=  df_yt1_gcp['cost']



df_yt1_gcp['processing_cost'].fillna(0,inplace=True)
df_yt1_gcp['storage_cost'].fillna(0,inplace=True)

<command-4462454918426130>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("[","")
<command-4462454918426130>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 df_yt1_gcp['credits'] = df_yt1_gcp['credits'].astype(str).str.replace("]","")
2

In [ ]:
df_yt1_gcp_agg = df_yt1_gcp[df_yt1_gcp['usage_start_time_LA'].astype(str).str.startswith(STR_YEARMONTH)].groupby(['usage_start_time_LA']).agg(total_GCP_cost = ('cost','sum')
                                                                ,total_processing_cost =('processing_cost','sum')
                                                                ,total_storage_cost =('storage_cost','sum')
                                                                ,total_GCP_discount = ('amount_credits','sum')
                                                                ,total_processing_discount =('processing_discount','sum')
                                                                ,total_storage_discount =('storage_discount','sum')
                                                                ,total_after_discounts =('total_after_discounts','sum')
                                                                ,total_processing_after_discount =('total_processing_after_discount','sum') ).reset_index()

In [ ]:
df_yt1_gcp_agg.loc[:,'total_GCP_cost_by_tenant'] = df_yt1_gcp_agg['total_GCP_cost']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_processing_cost_by_tenant'] = df_yt1_gcp_agg['total_processing_cost']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_GCP_discount_by_tenant'] = df_yt1_gcp_agg['total_GCP_discount']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_processing_discount_by_tenant'] = df_yt1_gcp_agg['total_processing_discount']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_storage_discount_by_tenant'] = df_yt1_gcp_agg['total_storage_discount']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_after_discounts_by_tenant'] = df_yt1_gcp_agg['total_after_discounts']/tenant_qty_yt1
df_yt1_gcp_agg.loc[:,'total_processing_after_discount_by_tenant'] = df_yt1_gcp_agg['total_processing_after_discount']/tenant_qty_yt1
df_yt1_gcp_agg.drop(columns ={'total_GCP_cost','total_storage_cost','total_processing_cost','total_GCP_discount','total_processing_discount','total_storage_discount','total_after_discounts','total_processing_after_discount'}, inplace = True)
df_yt1_gcp_agg.loc[:,'yearmonth']=df_yt1_gcp_agg['usage_start_time_LA'].astype(str).apply(lambda x: x[0:7])
df_yt1_gcp_agg_f = df_yt1_gcp_agg.groupby('yearmonth').sum().reset_index()

In [ ]:
df_yt1_gcp_agg_f

Out[25]:

,yearmonth,total_GCP_cost_by_tenant,total_processing_cost_by_tenant,total_GCP_discount_by_tenant,total_processing_discount_by_tenant,total_storage_discount_by_tenant,total_after_discounts_by_tenant,total_processing_after_discount_by_tenant
0,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803


In [ ]:
df_yts1_result = df_yt1_gcp_agg_f.merge(df_sites_yts1_s,how = 'inner', on = 'yearmonth')

In [ ]:
df_yts1_result.sum()['total_after_discounts_by_tenant']

Out[27]: 2892.001513000001

In [ ]:
df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("[","")
df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("]","")
df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("","")
try:
    df_vt1_gcp.loc[df_vt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = df_vt1_gcp['credits'].str.split('\n', expand=True)[0]
    df_vt1_gcp.loc[df_vt1_gcp['credits_1'].isnull(),'credits_1']=''
    json_cols = ['service', 'sku','usage','credits_1']
    try:
        df_vt1_gcp.loc[df_vt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_2'] = df_vt1_gcp['credits'].str.split('\n', expand=True)[1]
        df_vt1_gcp.loc[df_vt1_gcp['credits_2'].isnull(),'credits_2']=''
        json_cols = ['service', 'sku','usage','credits_1','credits_2']
    except:
        print ('2')
        json_cols = ['service', 'sku','usage','credits_1']
except:
    df_vt1_gcp.loc[df_vt1_gcp['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = df_vt1_gcp['credits'][0]
    df_vt1_gcp.loc[df_vt1_gcp['credits_1'].isnull(),'credits_1']=''
    json_cols = ['service', 'sku','usage','credits_1']


sku = df_vt1_gcp['sku'].values.tolist()
service = df_vt1_gcp['service'].values.tolist()

#json_cols = ['service', 'sku','usage','credits_1','credits_2']
for column in json_cols:
    #c_load = test[column].apply(json.loads)
    
    if(column == 'project'):  
        pr = pd.json_normalize(df_vt1_gcp['project'])
        pr.rename(columns = {'labels':'labels_%s'%(column)},inplace =True)
        df_vt1_gcp = df_vt1_gcp.join(pr)
        df_vt1_gcp.rename(columns = {'id':'id_%s'%(column)},inplace =True)
        df_vt1_gcp.rename(columns = {'labels':'description_%s'%(column)},inplace =True)
        df_vt1_gcp = df_vt1_gcp.drop(column , axis=1)
    if((column=='credits_1')|(column== 'credits_2')):
        #df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("[","")
        #df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("]","")
        #df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("","")
        df_vt1_gcp.loc[~(df_vt1_gcp[column].str.contains('1|2|3|4|5|6|7|8|9|0')),column] = "{'name':'','amount':0,'full_name':'','id':'', 'type':''}"
        t_er = list(df_vt1_gcp[column])
        t_er2 = json.dumps(t_er)
        t_er2 = t_er2.replace('"',"").replace("'",'"')
        if column == 'credits_1':
            df_credits = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits.rename(columns ={'amount':'amount_credits_1','id':'id_credits_1','name':'name_credits_1'
                                   ,'full_name':'full_name_credits_1', 'type':'type_credits_1'},inplace = True)
            df_vt1_gcp = df_vt1_gcp.join(df_credits)
            df_vt1_gcp.loc[df_vt1_gcp['type_credits_1']=='PROMOTION','promotion_credits_1'] = df_vt1_gcp['amount_credits_1']
            df_vt1_gcp.loc[df_vt1_gcp['type_credits_1']=='PROMOTION','amount_credits_1'] = 0
        if column == 'credits_2':
            df_credits_2 = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits_2.rename(columns ={'amount':'amount_credits_2','id':'id_credits_2','name':'name_credits_2'
                                   ,'full_name':'full_name_credits_2', 'type':'type_credits_2'},inplace = True)
            df_vt1_gcp = df_vt1_gcp.join(df_credits_2)
            df_credits_2.loc[df_credits_2['type_credits_2']=='PROMOTION','promotion_credits_2'] = df_credits_2['amount_credits_2']
            df_credits_2.loc[df_credits_2['type_credits_2']=='PROMOTION','amount_credits_2'] = 0
        
        df_vt1_gcp = df_vt1_gcp.drop(column , axis=1)

    if((column != 'project')&(column != 'credits_1')&(column != 'credits_2')):
        c_list = list(df_vt1_gcp[column])
        c_dat = json.dumps(c_list)    
        df_vt1_gcp = df_vt1_gcp.join(pd.read_json(c_dat))
        df_vt1_gcp.rename(columns = {'id':'id_%s'%(column)},inplace =True)
        df_vt1_gcp.rename(columns = {'description':'description_%s'%(column)},inplace =True)
        df_vt1_gcp = df_vt1_gcp.drop(column , axis=1)
try:        
  df_vt1_gcp.loc[:,'total_after_discounts']=  df_vt1_gcp['cost'] + df_vt1_gcp['amount_credits_1']+ df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[:,'amount_credits']= df_vt1_gcp['amount_credits_1']+ df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[(~(df_vt1_gcp['description_sku'].str.contains('SSD'))),'processing_discount']= df_vt1_gcp['amount_credits_1']+df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[(df_vt1_gcp['description_sku'].str.contains('SSD')),'storage_discount']= df_vt1_gcp['amount_credits_1']+df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[(~(df_vt1_gcp['description_sku'].str.contains('SSD'))),'total_processing_after_discount']=  df_vt1_gcp['cost']+df_vt1_gcp['amount_credits_1']+ df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[((df_vt1_gcp['description_sku'].str.contains('SSD'))),'total_storage_after_discount']=  df_vt1_gcp['cost']+df_vt1_gcp['amount_credits_1']+ df_vt1_gcp['amount_credits_2']
  df_vt1_gcp.loc[:,'promotion_credits'] = df_vt1_gcp['promotion_credits_1'].fillna(0) + df_vt1_gcp['promotion_credits_2'].fillna(0)
except:
  df_vt1_gcp.loc[:,'total_after_discounts']=  df_vt1_gcp['cost'] + df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[:,'amount_credits']= df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[(~(df_vt1_gcp['description_sku'].str.contains('SSD'))),'processing_discount']= df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[(df_vt1_gcp['description_sku'].str.contains('SSD')),'storage_discount']= df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[(~(df_vt1_gcp['description_sku'].str.contains('SSD'))),'total_processing_after_discount']=  df_vt1_gcp['cost']+df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[((df_vt1_gcp['description_sku'].str.contains('SSD'))),'total_storage_after_discount']=  df_vt1_gcp['cost']+df_vt1_gcp['amount_credits_1']
  df_vt1_gcp.loc[:,'promotion_credits'] = df_vt1_gcp['promotion_credits_1'].fillna(0) 

df_vt1_gcp.loc[(~(df_vt1_gcp['description_sku'].str.contains('SSD'))),'processing_cost']=  df_vt1_gcp['cost']
df_vt1_gcp.loc[(df_vt1_gcp['description_sku'].str.contains('SSD')),'storage_cost']=  df_vt1_gcp['cost']



df_vt1_gcp['processing_cost'].fillna(0,inplace=True)
df_vt1_gcp['storage_cost'].fillna(0,inplace=True)

<command-4462454918426129>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("[","")
<command-4462454918426129>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
 df_vt1_gcp['credits'] = df_vt1_gcp['credits'].astype(str).str.replace("]","")
2

In [ ]:
df_vt1_gcp_agg = df_vt1_gcp[df_vt1_gcp['usage_start_time_LA'].astype(str).str.startswith(STR_YEARMONTH)].groupby(['usage_start_time_LA']).agg(total_GCP_cost = ('cost','sum')
                                                                ,total_processing_cost =('processing_cost','sum')
                                                                ,total_storage_cost =('storage_cost','sum')
                                                                ,total_GCP_discount = ('amount_credits','sum')
                                                                ,total_processing_discount =('processing_discount','sum')
                                                                ,total_storage_discount =('storage_discount','sum')
                                                                ,total_after_discounts =('total_after_discounts','sum')
                                                                ,total_processing_after_discount =('total_processing_after_discount','sum') ).reset_index()

In [ ]:
df_vt1_gcp_agg.loc[:,'total_GCP_cost_by_tenant'] = df_vt1_gcp_agg['total_GCP_cost']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_processing_cost_by_tenant'] = df_vt1_gcp_agg['total_processing_cost']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_GCP_discount_by_tenant'] = df_vt1_gcp_agg['total_GCP_discount']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_processing_discount_by_tenant'] = df_vt1_gcp_agg['total_processing_discount']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_storage_discount_by_tenant'] = df_vt1_gcp_agg['total_storage_discount']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_after_discounts_by_tenant'] = df_vt1_gcp_agg['total_after_discounts']/tenant_qty_vt1
df_vt1_gcp_agg.loc[:,'total_processing_after_discount_by_tenant'] = df_vt1_gcp_agg['total_processing_after_discount']/tenant_qty_vt1
df_vt1_gcp_agg.drop(columns ={'total_GCP_cost','total_storage_cost','total_processing_cost','total_GCP_discount','total_processing_discount','total_storage_discount','total_after_discounts','total_processing_after_discount'}, inplace = True)
df_vt1_gcp_agg.loc[:,'yearmonth']=df_vt1_gcp_agg['usage_start_time_LA'].astype(str).apply(lambda x: x[0:7])
df_vt1_gcp_agg_f = df_vt1_gcp_agg.groupby('yearmonth').sum().reset_index()

In [ ]:
df_vts1_result = df_vt1_gcp_agg_f.merge(df_sites_vts1_s,how = 'inner', on = 'yearmonth')

In [ ]:
df_vts1_result.sum()['total_after_discounts_by_tenant']

Out[32]: 917.7665270000001

In [ ]:
df_vts1_result

Out[33]:

,yearmonth,total_GCP_cost_by_tenant,total_processing_cost_by_tenant,total_GCP_discount_by_tenant,total_processing_discount_by_tenant,total_storage_discount_by_tenant,total_after_discounts_by_tenant,total_processing_after_discount_by_tenant,tenant_parent_code,valid_tenant
0,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,IKEA,1.0
1,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,HONDA,1.0
2,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,BOLZ,1.0
3,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,FEDEX,1.0


In [ ]:
df_yts1_result.loc[:,'environment']='yt1'
df_vts1_result.loc[:,'environment']='vt1'

In [ ]:
df_result_all = df_vts1_result.append(df_yts1_result)

In [ ]:
df_result_all

Out[36]:

,yearmonth,total_GCP_cost_by_tenant,total_processing_cost_by_tenant,total_GCP_discount_by_tenant,total_processing_discount_by_tenant,total_storage_discount_by_tenant,total_after_discounts_by_tenant,total_processing_after_discount_by_tenant,tenant_parent_code,valid_tenant,environment
0,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,IKEA,1.0,vt1
1,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,HONDA,1.0,vt1
2,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,BOLZ,1.0,vt1
3,2023-02,232.644539,202.384781,-3.202907,-3.202907,0.0,229.441632,199.181873,FEDEX,1.0,vt1
0,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,FALAPE,1.0,yt1
1,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,OECH,1.0,yt1
2,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,CENT,1.0,yt1
3,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,FALACO,1.0,yt1
4,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,IKEA,1.0,yt1
5,2023-02,180.977546,171.627599,-20.310795,-20.310795,0.0,160.666751,151.316803,FISIA,1.0,yt1


In [ ]:
table = "saas-analytics-io.processed.sandbox_cost_allocation_y1_vt1"
df_result_all.to_gbq(table,  if_exists='append')